# TÍNH TOÁN GRADIENT TRONG TORCH.TENSOR (Code for Pytorch)
## Import các Libarary
Sử dụng hai thư viện thao tác với matrix và vector là _numpy_ và _torch_.

In [1]:
import numpy as np
import torch as T
import torch.nn as nn
import torch.nn.functional as F

## Gradient trong Tensor
Thiết lập một bộ số random cố định cho Tensor và tạo một mạng <font color='red'>_NN1_</font> với 2 ngõ vào và một ngõ ra.
_NN1_ có tham số Weigh $\boldsymbol{\theta}=[\theta_1,\theta_2]$ và Bias $b$. Một mini-batch _in_var_ có batch-size $B=3$, $\textbf{X}=\{\textbf{x}_1,\textbf{x}_1,\textbf{x}_3\}$ với $\textbf{x}_n=[x_{n,1},x_{n,2}]^\top, n=1,2,3,$ được đựa vào mạng và hàm _loss_ được tính bởi:
$$
loss_n=NN1(\textbf{x}_n)
$$
Một cách chi tiết hàm _loss_ được viết lại
$$
loss=\theta_1 x_{n,1}+\theta_2 x_{n,2}+b
$$
Như vậy _Gradient_ w.r.t $\boldsymbol{\theta}$, $\nabla_{\boldsymbol{\theta}}|_{\textbf{x}_n}$, chính là $\textbf{x}_n^\top$. Và _Gradient_ cho toàn mini-batch sẽ là trung bình của các _Gradient_ thành phần 
$$
\nabla_{\boldsymbol{\theta}}^\text{mini-batch}=\frac{1}{B}\sum\limits_{n=1}^3 \textbf{x}_n=\left[\frac{1.4}{3},0.2\right]
$$

In [3]:
T.manual_seed(10)
in_var = T.tensor([[0.5,-0.6],[0.7,1.],[0.2,0.2]],requires_grad=True)
NN1 = nn.Linear(2,1)
# nn.init.uniform_(NN1.weight.data,-1,1) # set param with rules
# nn.init.constant_(NN1.bias.data,-1)
out_var = NN1(in_var)

loss = T.mean(out_var)
print(loss)
NN1.zero_grad()
loss.backward()
print(NN1.weight.grad)

tensor(-0.2977, grad_fn=<MeanBackward0>)
tensor([[0.4667, 0.2000]])


## Ảnh hưởng của Detach và Numpy đối với Gradient
Ở phần này _loss_ sẽ được tính toán thủ công thông qua thao tác trên _numpy_ và sau đó kiểm tra _Gradient_ của $\boldsymbol{\theta}$.
Đầu tiên, ta _detach()_ ngõ ra của _NN1_ (chính là _out_var_) sao đó chuyển sang _numpy_. Từ đây _mean_ được tính thủ công. Sau đó, _loss_ được chuyển về dạng _tensor_ tương ứng với dạng _output_ của phép <font color=red>_T.mean(out_var)_</font>. Tương tự ta tính _Gradient_ và thấy $
\nabla_{\boldsymbol{\theta}}^\text{mini-batch}=\left[0,0\right]
$. Như vậy, khi thao tác với _tensor_ phải chú ý việc bảo toàn _Gradient_ ở mỗi bước tính toán.

In [4]:
out_var = out_var.detach()
out_var = out_var.numpy()
lost_1 = out_var.sum()/out_var.shape[0]
lost_1 = T.tensor(lost_1,requires_grad=True)
NN1.zero_grad()
lost_1.backward()
print(NN1.weight.grad)

tensor([[0., 0.]])


In [ ]:
## Ảnh hưởng Batch-Norm-1D đối với Gradient


In [ ]:
# elif check_effect_detach == 2:
#     # khong detach, thong tin grad van co n
# in_var = T.tensor([[1.,2.],[0.,-1.]],requires_grad=True)

# print(in_var.shape[0])
# if in_var.shape[0]>1:
#     BN1 = nn.BatchNorm1d(1)
#     out_var_bn = BN1(out_var)
#     print(out_var_bn)
#     out_var_bn = T.mean(out_var_bn)
#     print(out_var_bn)
#     out_var_bn.backward()
#     print(NN1.weight.grad)
    
# pointer_param_NN1  = NN1.named_parameters()
# print(dict(pointer_param_NN1))    
# NN2 = nn.Linear(2,1)
# NN2.load_state_dict(dict(NN1.named_parameters()))
# print(NN1.weight.data)
# print(NN2.weight.data)